# Full Dataset Visualization

This notebook visualizes trajectory predictions for all universities in the dataset using PCA for dimensionality reduction and interactive Plotly charts.

## 1. Setup & Imports

In [216]:
import pandas as pd
import numpy as np
import joblib
from pathlib import Path

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Paths
NOTEBOOK_DIR = Path().resolve()
ASSETS_DIR = NOTEBOOK_DIR.parent.parent / "assets"
MODEL_PATH = ASSETS_DIR / "models" / "trajectory_model.joblib"
DATA_PATH = ASSETS_DIR / "data" / "trajectory_excellent.csv"
OUTPUT_DIR = NOTEBOOK_DIR.parent / "images"

print(f"Model path: {MODEL_PATH}")
print(f"Data path: {DATA_PATH}")
print(f"Output dir: {OUTPUT_DIR}")

Model path: /Users/mukeshravichandran/ML EDA/final/assets/models/trajectory_model.joblib
Data path: /Users/mukeshravichandran/ML EDA/final/assets/data/trajectory_excellent.csv
Output dir: /Users/mukeshravichandran/ML EDA/final/postanalysis/images


## 2. Load Model & Dataset

In [217]:
# Load trained XGBoost model
model = joblib.load(MODEL_PATH)
print(f"✅ Model loaded: {type(model).__name__}")

# Load full dataset
df_full = pd.read_csv(DATA_PATH)
print(f"✅ Dataset loaded: {len(df_full):,} rows, {df_full['UNITID'].nunique():,} unique schools")
print(f"   Years covered: {df_full['Year'].min():.0f} - {df_full['Year'].max():.0f}")

✅ Model loaded: Pipeline
✅ Dataset loaded: 10,332 rows, 1,722 unique schools
   Years covered: 2017 - 2022


## 3. Filter to Latest Year Per School

We want ONE prediction per university — using their most recent data.

In [218]:
# Get the latest year for each school
idx_latest = df_full.groupby('UNITID')['Year'].idxmax()
df = df_full.loc[idx_latest].copy().reset_index(drop=True)

print(f"✅ Filtered to latest year per school: {len(df):,} universities")
print(f"   Year distribution:")
print(df['Year'].value_counts().sort_index())

✅ Filtered to latest year per school: 1,722 universities
   Year distribution:
Year
2022    1722
Name: count, dtype: int64


## 4. Run Batch Predictions

In [219]:
# Columns to drop before prediction (identifiers and targets)
DROP_COLS = ['UNITID', 'Institution_Name', 'Year', 'Target_Trajectory', 'Target_Label', 'State']

# Prepare features
feature_cols = [c for c in df.columns if c not in DROP_COLS]
X = df[feature_cols].copy()

print(f"Feature matrix shape: {X.shape}")
print(f"Features: {len(feature_cols)} columns")

Feature matrix shape: (1722, 46)
Features: 46 columns


In [236]:
# Run predictions
predictions = model.predict(X)
probabilities = model.predict_proba(X)

# Add to dataframe
df['Predicted_Class'] = predictions
df['Predicted_Label'] = df['Predicted_Class'].map({0: 'Declining', 1: 'Stable', 2: 'Improving'})

# Add probabilities
df['Prob_Declining'] = probabilities[:, 0]
df['Prob_Stable'] = probabilities[:, 1]
df['Prob_Improving'] = probabilities[:, 2]
df['Confidence'] = probabilities.max(axis=1)

# Target_Trajectory already contains string labels ('Stable', 'Declining', 'Improving')
df['Actual_Label'] = df['Target_Trajectory']

print("✅ Predictions complete!")
print(f"\nPredicted distribution:")
print(df['Predicted_Label'].value_counts())
print(f"\nActual distribution:")
print(df['Actual_Label'].value_counts())

✅ Predictions complete!

Predicted distribution:
Predicted_Label
Stable       947
Declining    520
Improving    255
Name: count, dtype: int64

Actual distribution:
Actual_Label
Stable       972
Declining    487
Improving    263
Name: count, dtype: int64


## 5. Apply PCA for 2D Visualization

We reduce numerical features to 2 principal components for scatter plot positioning.

In [221]:
# Select only numeric features for PCA
numeric_cols = X.select_dtypes(include=[np.number]).columns.tolist()
X_numeric = X[numeric_cols].fillna(0)

print(f"Numeric features for PCA: {len(numeric_cols)}")

# Standardize features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_numeric)

# Apply PCA
pca = PCA(n_components=2, random_state=42)
pca_result = pca.fit_transform(X_scaled)

df['PC1'] = pca_result[:, 0]
df['PC2'] = pca_result[:, 1]

print(f"\n✅ PCA complete!")
print(f"   Explained variance: PC1={pca.explained_variance_ratio_[0]:.1%}, PC2={pca.explained_variance_ratio_[1]:.1%}")
print(f"   Total explained: {sum(pca.explained_variance_ratio_):.1%}")

Numeric features for PCA: 44

✅ PCA complete!
   Explained variance: PC1=35.7%, PC2=12.1%
   Total explained: 47.8%

✅ PCA complete!
   Explained variance: PC1=35.7%, PC2=12.1%
   Total explained: 47.8%


## 6. Identify Highlighted Schools

Mark the 4 universities from previous prediction reports with special symbols.

In [222]:
# Schools to highlight
HIGHLIGHT_SCHOOLS = [
    'University of Texas at Austin',
    'Texas A & M University-College Station',
    'Texas A & M University-Commerce',
    'University of Alabama'
]

# Find matches (partial matching)
def find_school(name, df):
    matches = df[df['Institution_Name'].str.contains(name, case=False, na=False)]
    if len(matches) > 0:
        return matches.iloc[0]
    return None

highlighted = []
for school in HIGHLIGHT_SCHOOLS:
    match = find_school(school, df)
    if match is not None:
        highlighted.append({
            'name': match['Institution_Name'],
            'PC1': match['PC1'],
            'PC2': match['PC2'],
            'predicted': match['Predicted_Label'],
            'actual': match['Actual_Label'],
            'confidence': match['Confidence']
        })
        print(f"✅ Found: {match['Institution_Name']} → {match['Predicted_Label']} ({match['Confidence']:.1%})")

df_highlighted = pd.DataFrame(highlighted)

✅ Found: The University of Texas at Austin → Improving (99.9%)
✅ Found: Texas A & M University-College Station → Declining (77.0%)
✅ Found: Texas A & M University-Commerce → Improving (57.5%)
✅ Found: University of Alabama at Birmingham → Stable (62.7%)


## 7. Interactive Scatter Plot — Predicted Trajectory

Each dot is a university. Hover to see details. Highlighted schools marked with ★.

In [223]:
# Color mapping
color_map = {
    'Declining': '#E74C3C',   # Red
    'Stable': '#F4D03F',      # Gold/Yellow
    'Improving': '#27AE60'    # Green
}

# Create hover text
df['hover_text'] = (
    '<b>' + df['Institution_Name'] + '</b><br>' +
    'State: ' + df['State'].astype(str) + '<br>' +
    'Division: ' + df['Division'].astype(str) + '<br>' +
    '<br><b>Predicted:</b> ' + df['Predicted_Label'] + '<br>' +
    'Confidence: ' + (df['Confidence'] * 100).round(1).astype(str) + '%<br>' +
    '<br>Declining: ' + (df['Prob_Declining'] * 100).round(1).astype(str) + '%<br>' +
    'Stable: ' + (df['Prob_Stable'] * 100).round(1).astype(str) + '%<br>' +
    'Improving: ' + (df['Prob_Improving'] * 100).round(1).astype(str) + '%'
)

# Main scatter plot
fig = px.scatter(
    df,
    x='PC1',
    y='PC2',
    color='Predicted_Label',
    color_discrete_map=color_map,
    hover_name='Institution_Name',
    custom_data=['hover_text'],
    title='<b>University Financial Trajectory Classification</b><br><sup>All Universities Colored by Predicted Trajectory (Latest Year)</sup>',
    labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2'},
    category_orders={'Predicted_Label': ['Declining', 'Stable', 'Improving']}
)

# Update hover template
fig.update_traces(
    hovertemplate='%{customdata[0]}<extra></extra>',
    marker=dict(size=8, opacity=0.7, line=dict(width=0.5, color='white'))
)

# Add highlighted schools as stars
if len(df_highlighted) > 0:
    fig.add_trace(go.Scatter(
        x=df_highlighted['PC1'],
        y=df_highlighted['PC2'],
        mode='markers+text',
        marker=dict(size=20, symbol='star', color='black', line=dict(width=2, color='white')),
        text=['★'] * len(df_highlighted),
        textposition='middle center',
        textfont=dict(size=14, color='white'),
        name='Highlighted Schools',
        hovertemplate=(
            '<b>%{customdata[0]}</b><br>' +
            'Predicted: %{customdata[1]}<br>' +
            'Confidence: %{customdata[2]:.1%}<extra></extra>'
        ),
        customdata=df_highlighted[['name', 'predicted', 'confidence']].values
    ))

# Layout
fig.update_layout(
    width=1000,
    height=700,
    template='plotly_white',
    legend_title_text='Predicted Trajectory',
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=0.01)
)

fig.show()

In [224]:
# Save the interactive plot
fig.write_html(OUTPUT_DIR / 'university_trajectory_scatter.html')
fig.write_image(OUTPUT_DIR / 'university_trajectory_scatter.png', scale=2)
print(f"✅ Saved: {OUTPUT_DIR / 'university_trajectory_scatter.html'}")
print(f"✅ Saved: {OUTPUT_DIR / 'university_trajectory_scatter.png'}")

✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/university_trajectory_scatter.html
✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/university_trajectory_scatter.png


## 8. Side-by-Side: Actual vs Predicted

Compare how the model's predictions align with actual trajectories.

In [225]:
# Create side-by-side subplots
fig_compare = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Actual Trajectory', 'Predicted Trajectory'),
    horizontal_spacing=0.1
)

# Order for consistent coloring
trajectory_order = ['Declining', 'Stable', 'Improving']

# LEFT: Actual trajectory
for label in trajectory_order:
    mask = df['Actual_Label'] == label
    fig_compare.add_trace(
        go.Scatter(
            x=df.loc[mask, 'PC1'],
            y=df.loc[mask, 'PC2'],
            mode='markers',
            marker=dict(size=6, color=color_map[label], opacity=0.7),
            name=f'{label} (Actual)',
            legendgroup=label,
            showlegend=True,
            hovertemplate=df.loc[mask, 'Institution_Name'] + '<br>Actual: ' + label + '<extra></extra>'
        ),
        row=1, col=1
    )

# RIGHT: Predicted trajectory
for label in trajectory_order:
    mask = df['Predicted_Label'] == label
    fig_compare.add_trace(
        go.Scatter(
            x=df.loc[mask, 'PC1'],
            y=df.loc[mask, 'PC2'],
            mode='markers',
            marker=dict(size=6, color=color_map[label], opacity=0.7),
            name=f'{label} (Predicted)',
            legendgroup=label,
            showlegend=False,
            hovertemplate=df.loc[mask, 'Institution_Name'] + '<br>Predicted: ' + label + '<extra></extra>'
        ),
        row=1, col=2
    )

# Layout
fig_compare.update_layout(
    title='<b>Actual vs Predicted Trajectory Comparison</b><br><sup>How well does the model separate the classes?</sup>',
    width=1200,
    height=600,
    template='plotly_white',
    legend=dict(yanchor='top', y=0.99, xanchor='left', x=1.02)
)

fig_compare.update_xaxes(title_text='PC1', row=1, col=1)
fig_compare.update_xaxes(title_text='PC1', row=1, col=2)
fig_compare.update_yaxes(title_text='PC2', row=1, col=1)
fig_compare.update_yaxes(title_text='PC2', row=1, col=2)

fig_compare.show()

In [226]:
# Save comparison plot
fig_compare.write_html(OUTPUT_DIR / 'actual_vs_predicted_comparison.html')
fig_compare.write_image(OUTPUT_DIR / 'actual_vs_predicted_comparison.png', scale=2)
print(f"✅ Saved: {OUTPUT_DIR / 'actual_vs_predicted_comparison.html'}")
print(f"✅ Saved: {OUTPUT_DIR / 'actual_vs_predicted_comparison.png'}")

✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/actual_vs_predicted_comparison.html
✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/actual_vs_predicted_comparison.png


## 9. Distribution Bar Chart

In [227]:
# Count per category
dist_data = df['Predicted_Label'].value_counts().reindex(['Declining', 'Stable', 'Improving'])

fig_dist = px.bar(
    x=dist_data.index,
    y=dist_data.values,
    color=dist_data.index,
    color_discrete_map=color_map,
    title='<b>Predicted Trajectory Distribution</b><br><sup>Number of Universities in Each Category</sup>',
    labels={'x': 'Trajectory', 'y': 'Number of Universities'},
    text=dist_data.values
)

fig_dist.update_traces(textposition='outside', textfont_size=14)
fig_dist.update_layout(
    width=700,
    height=500,
    template='plotly_white',
    showlegend=False,
    xaxis_title='Predicted Trajectory',
    yaxis_title='Count'
)

# Add percentage annotations
total = len(df)
for i, (label, count) in enumerate(dist_data.items()):
    pct = count / total * 100
    fig_dist.add_annotation(
        x=label, y=count + 30,
        text=f'{pct:.1f}%',
        showarrow=False,
        font=dict(size=12, color='gray')
    )

fig_dist.show()

In [228]:
# Save distribution chart
fig_dist.write_html(OUTPUT_DIR / 'trajectory_distribution.html')
fig_dist.write_image(OUTPUT_DIR / 'trajectory_distribution.png', scale=2)
print(f"✅ Saved: {OUTPUT_DIR / 'trajectory_distribution.html'}")
print(f"✅ Saved: {OUTPUT_DIR / 'trajectory_distribution.png'}")

✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/trajectory_distribution.html
✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/trajectory_distribution.png


## 10. Distribution by Division

In [229]:
# Cross-tabulation
division_dist = pd.crosstab(df['Division'], df['Predicted_Label'])
division_dist = division_dist[['Declining', 'Stable', 'Improving']]

fig_div = px.bar(
    division_dist,
    barmode='group',
    color_discrete_map=color_map,
    title='<b>Predicted Trajectory by NCAA Division</b>',
    labels={'value': 'Number of Universities', 'Division': 'NCAA Division'},
)

fig_div.update_layout(
    width=900,
    height=500,
    template='plotly_white',
    legend_title_text='Trajectory',
    xaxis_title='NCAA Division',
    yaxis_title='Count'
)

fig_div.show()

In [230]:
# Save division chart
fig_div.write_html(OUTPUT_DIR / 'trajectory_by_division.html')
fig_div.write_image(OUTPUT_DIR / 'trajectory_by_division.png', scale=2)
print(f"✅ Saved: {OUTPUT_DIR / 'trajectory_by_division.html'}")
print(f"✅ Saved: {OUTPUT_DIR / 'trajectory_by_division.png'}")

✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/trajectory_by_division.html
✅ Saved: /Users/mukeshravichandran/ML EDA/final/postanalysis/images/trajectory_by_division.png


## 11. Save All Predictions to CSV

In [231]:
# Select columns for export
export_cols = [
    'UNITID', 'Institution_Name', 'State', 'Division', 'Year',
    'Actual_Label', 'Predicted_Label', 'Confidence',
    'Prob_Declining', 'Prob_Stable', 'Prob_Improving',
    'PC1', 'PC2'
]

df_export = df[export_cols].copy()
df_export = df_export.sort_values('Institution_Name')

output_csv = NOTEBOOK_DIR.parent / 'all_predictions.csv'
df_export.to_csv(output_csv, index=False)

print(f"✅ Saved predictions: {output_csv}")
print(f"   Total universities: {len(df_export):,}")
print(f"\nSample:")
df_export.head(10)

✅ Saved predictions: /Users/mukeshravichandran/ML EDA/final/postanalysis/all_predictions.csv
   Total universities: 1,722

Sample:


,UNITID,Institution_Name,State,Division,Year,Actual_Label,Predicted_Label,Confidence,Prob_Declining,Prob_Stable,Prob_Improving,PC1,PC2
1444,222178,Abilene Christian University,TX,D1,2022,NaN,Stable,0.999873,0.000048,0.999873,0.000079,1.401265,-1.442694
294,138558,Abraham Baldwin Agricultural College,GA,Other,2022,NaN,Improving,0.562499,0.001995,0.435506,0.562499,-1.949211,-1.538011
81,108232,Academy of Art University,CA,D1,2022,NaN,Stable,0.999815,0.000070,0.999815,0.000116,-0.622530,-1.584204
190,126182,Adams State University,CO,D1,2022,NaN,Stable,0.999896,0.000050,0.999896,0.000054,-0.750542,-1.457335
961,188429,Adelphi University,NY,D1,2022,NaN,Stable,0.999887,0.000030,0.999887,0.000083,-0.051545,2.104941
690,168528,Adrian College,MI,D1,2022,NaN,Stable,0.999896,0.000046,0.999896,0.000058,-0.867141,-1.991479
0,100654,Alabama A & M University,AL,D1,2022,NaN,Stable,0.999754,0.000071,0.999754,0.000175,-0.306323,-1.953912
3,100724,Alabama State University,AL,D1,2022,NaN,Declining,0.901688,0.901688,0.000832,0.097480,-0.026093,-1.577633
963,188526,Albany College of Pharmacy and Health Sciences,NY,Other,2022,NaN,Stable,0.832667,0.000304,0.832667,0.167029,-1.906375,5.274394
296,138716,Albany State University,GA,D1,2022,NaN,Declining,0.999706,0.999706,0.000177,0.000118,-1.284216,0.455781


## 12. Summary Statistics

In [237]:
# Model accuracy on this subset
correct = (df['Predicted_Label'] == df['Actual_Label']).sum()
accuracy = correct / len(df)

print("=" * 50)
print("SUMMARY: Full Dataset Trajectory Predictions")
print("=" * 50)
print(f"\nTotal Universities: {len(df):,}")
print(f"Latest Year Used: {df['Year'].mode().iloc[0]:.0f}")
print(f"\nPredicted Distribution:")
for label in ['Declining', 'Stable', 'Improving']:
    count = (df['Predicted_Label'] == label).sum()
    pct = count / len(df) * 100
    print(f"  {label:12}: {count:5} ({pct:5.1f}%)")

print(f"\nModel Accuracy (on latest year): {accuracy:.1%}")
print(f"Correct predictions: {correct:,} / {len(df):,}")

print(f"\nHighlighted Schools:")
for _, row in df_highlighted.iterrows():
    match = "✓" if row['actual'] == row['predicted'] else "✗"
    print(f"  {match} {row['name'][:40]:40} → {row['predicted']} ({row['confidence']:.1%})")

print("\n" + "=" * 50)
print("Files saved to: postanalysis/images/")
print("=" * 50)

SUMMARY: Full Dataset Trajectory Predictions

Total Universities: 1,722
Latest Year Used: 2022

Predicted Distribution:
  Declining   :   520 ( 30.2%)
  Stable      :   947 ( 55.0%)
  Improving   :   255 ( 14.8%)

Model Accuracy (on latest year): 92.8%
Correct predictions: 1,598 / 1,722

Highlighted Schools:
  ✗ The University of Texas at Austin        → Improving (99.9%)
  ✗ Texas A & M University-College Station   → Declining (77.0%)
  ✗ Texas A & M University-Commerce          → Improving (57.5%)
  ✗ University of Alabama at Birmingham      → Stable (62.7%)

Files saved to: postanalysis/images/
